# Clustering and the curse of dimensionality










**Ariel Rossanigo**


### Quien soy?

* Ariel Rossanigo
* Artificial Intelligence teacher at UCSE-DAR
* Developer, Data Scientist
* Co-Founder of Bloom AI

### Clustering 

<div><img src="./imgs/clustering.jpg" width="50%" style="float: right; margin: 10px;" align="middle"></div>


#### *Encontrar grupos de datos que son similares*




* La definicion parece simple... pero hay que ver que consideramos grupos y que consideramos similares 


In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from under_the_carpet import *

data = np.load('clusterable_data.npy')

In [ ]:
plot_data(data)

In [ ]:
%time kmeans = cluster.KMeans(n_clusters=6).fit(data)
plot_data(data, kmeans.predict(data), legend=False, title='K-Means', centroids=kmeans.cluster_centers_)

* Como funciona
* Ventajas: Converge rapido
* Desventajas: Hay que elegir K; cluster esféricos; todas las instancias pertenecen a un cluster

In [ ]:
%time affinity = cluster.AffinityPropagation(damping=0.985, max_iter=200).fit(data)
plot_data(data, affinity.labels_, title='Affinity propagation', legend=False, centroids=affinity.cluster_centers_)

* Como funciona
* Ventajas: No hay que elegir K
* Desventajas: Hay que elegir damping; cluster esféricos; todas las instancias pertenecen a un cluster; Lento

In [ ]:
%time ms = cluster.MeanShift(bandwidth=0.2, cluster_all=False).fit(data)
plot_data(data, ms.labels_, title='Mean Shift', legend=False, centroids=ms.cluster_centers_)

* Como funciona
* Ventajas: No hay que elegir K, No todas las instancias pertenecen a un cluster 
* Desventajas: Hay que elegir otros parametros; cluster esféricos; Lento

In [ ]:
%time agg = cluster.AgglomerativeClustering(n_clusters=None, distance_threshold=2).fit(data)
plot_data(data, agg.labels_, title='Agglomerative clustering', legend=False)

* Como funciona
* Ventajas: Algunas formas de elegir que cluster fusionar no restringen la forma de los clusters
* Desventajas: Hay que elegir donde cortar, es el equivalente a elegir el K; si los datasets son grandes, la cantidad de informacion es abrumadora.

In [ ]:
%time dbscan = cluster.DBSCAN(eps=0.02, min_samples=5).fit(data)
plot_data(data, dbscan.labels_, title='DBSCAN clustering', legend=False)

<div><img src="./imgs/dbscan.png" width="40%" style="float: left; margin: 10px;" align="middle"></div>


<div><img src="./imgs/dbscan_working.gif" width="50%" style="float: right; margin: 10px;" align="middle"></div>


* Como funciona
 * Tiene core points, border points y outliers
 * Dos parametros importantes: epsilon y min_samples
* Ventajas: Encuentra areas densas y elimina outliers
* Desventajas: Elegir los parametros es dificil si hay mas de 2 dimensiones

### Clustering 

* El algoritmo de clustering depende mucho de que necesitamos
* En muchas aplicaciones, es encontrar regiones densas de casos en medio de ruido
* La opción por defecto quizas debiera ser basada en densidad 

### Curse of dimensionality

<div><img src="./imgs/sphere.png" width="40%" style="float: left; margin: 10px;" align="middle"></div>


<div><img src="./imgs/curse_in_distance.png" width="50%" style="float: right; margin: 10px;" align="middle"></div>


* El volumen del espacio se incremente, los datos quedan dispersos. Baja significancia estadistica. Ej de promedio con un solo campo categorico y luego con dos.
* Particularmente en clustering, hace que las distancias entre puntos tiendan a ser todas similares
* Otro no menor, se complica "mirar" los datos 

### Reducción de dimensionalidad

*Find the latent features in your data*


* Por que?
 * Quiero visualizar mis datos
 * Reducir redundancia
 
* Hay dos formas de reducir la dimensionalidad:
 * Feature selection
 * Feature projection

In [ ]:
digits_X, digits_y = load_mnist(30_000)

for index, (image, label) in enumerate(zip(digits_X[:32], digits_y[:32])):
    plt.subplot(4, 8, index + 1)
    plt.axis('off')
    plt.imshow(image.reshape(28, 28), cmap=plt.cm.gray_r, interpolation='nearest')

### PCA

<div><img src="./imgs/pca.png" width="80%" style="float: left; margin: 10px;" align="middle"></div>

* Es rapido
* Puede capturar solo correlaciones lineales entre variables

In [ ]:
%time emb = PCA(n_components=2).fit_transform(digits_X)
plot_data(emb, labels=digits_y, title='PCA a dos dimensiones')

### UMAP

<div><img src="./imgs/umap.png" width="100%" style="float: left; margin: 10px;" align="middle"></div>

* Consiste de dos etapas: encontrar un grafo y luego optimizar una representacion en menos dimensiones
* Tiene algunas ventajas:
 * Mantiene las relaciones globales
 * Sirve para hacer visualizaciones (trata de separar los clusters)
 * se puede usar como paso previo a un algoritmo de clusting (se puede facilmente generar varias dimensiones)
 * Se puede hacer reduccion de dimensionalidad supervisada

In [ ]:
%time emb = UMAP().fit_transform(digits_X)
plot_data(emb, labels=digits_y, title='UMAP a dos dimensiones')

In [ ]:
%time emb = UMAP().fit_transform(digits_X, y=digits_y)
plot_data(emb, labels=digits_y, title='UMAP supervisado a dos dimensiones')

#### Realidad ¿?

<div><img src="./imgs/dd.jpg" width="60%" style="float: right; margin: 10px;"  align="middle"></div>

* El ejemplo (creado de forma ficticia) consta de personajes que se usaron para jugar D&D.
* Cada personaje esta caracterizado por su raza, altura, peso, edad, y los puntos extra que tiene en algunas habilidades(fortaleza, carisma y constitucion)
* El dataset fue creado siguiendo algunas estadisticas que encontre por internet, por ejemplo:
 * la distribución de las razas que se utilizan para jugar es real
 * los enanos miden entre 4'2'' y 4'8'', pesan... etc.
 * la idea seria ver si los algoritmos que vimos son capaces de detectar los grupos generados por las distintas clases (9 en total)
 
* Tenemos una columna most_common_race que tiene un 1 para las 3 razas mas comunes

In [ ]:
characters = load_dd(30_000)
characters.head()

In [ ]:
characters.race.value_counts()

In [ ]:
dfm = DataFrameMapper([
    (['height'], StandardScaler()),
    (['weight'], StandardScaler()),
    (['age'], StandardScaler()),
    (['strength'], None),
    (['charisma'], None),
    (['constitution'], None),
])

X = dfm.fit_transform(characters)
y = characters.race
%time emb = UMAP(random_state=42).fit_transform(X)

Lo primero que vamos a ver es si podemos visualizar los datos...

In [ ]:
plot_data(emb, y, title='UMAP with informative features')

In [ ]:
noise_columns = []
for i in range(100):
    col = f'noisy_{i}'
    characters[col] = np.random.random(size=len(characters))
    noise_columns.append(col)

Ahora vamos a tentar a la suerte, dijimos que si hay muchas dimensiones con ruido

In [ ]:
dfm = DataFrameMapper([
    (['height'], StandardScaler()),
    (['weight'], StandardScaler()),
    (['age'], StandardScaler()),
    (['strength'], None),
    (['charisma'], None),
    (['constitution'], None),
    (noise_columns, StandardScaler())
])

X = dfm.fit_transform(characters)
y = characters.race

%time embed = UMAP(random_state=42).fit_transform(X)

In [ ]:
plot_data(embed, y)

Agregando muchas columnas con ruido ya no se distingue los clusters

In [ ]:
y2 = characters.most_common_race

%time embed = UMAP(random_state=42).fit_transform(X, y=y2)
plot_data(embed, y)

* Incluso en modo supervisado no es capaz de generar los clusters que teniamos al inicio
* Debieramos eliminar el ruido, quizas eliminando dimensiones. Vamos a intentar hacerlo usando Arboles de decision

### Forest embeddings

*Usar las hojas de los arboles como embedding de los datos*

* La idea es entrenar un ensemble de arboles, usando como target el campo de razas comunes.
* Luego a cada ejemplo lo pasamos por el ensemble y nos quedamos con el id de hoja de cada uno de los arboles
* Lo esperable es que ejemplos parecidos vayan "cayendo" en las mismas hojas de los arboles.
* Si hay dos subgrupos diferentes para un valor del target, podriamos suponer que pasarian por distintos caminos en cada arbol.

* Luego aplicariamos la reduccion de dimensionalidad sobre el listado de hojas en lugar de usar las features originales

In [ ]:
et = ExtraTreesClassifier(n_estimators=100, min_samples_leaf=3000, max_features=0.8, 
                          bootstrap=True, class_weight='balanced', max_leaf_nodes=10)

skf = StratifiedKFold(n_splits=2, shuffle=True)
preds = cross_val_predict(et, X, y2, cv=skf, method='predict_proba')

print('Area under the ROC Curve:', metrics.roc_auc_score(y2, preds[:,1]))

Medimos que el ensemble no este sobre-entrenando

In [ ]:
et.fit(X, y=y2)
leaves = et.apply(X)
leaves

* Entrenamos el arbol con todos los datos
* Vemos que para cada ejemplo tenemos el array de indices que indica en que hoja cayo ese ejemplo para cada arbol del ensemble

In [ ]:
%time umap_reducer = UMAP(metric='hamming').fit(leaves)
final_emb = umap_reducer.transform(leaves)
plot_data(final_emb, y)

Usamos Hamming distance porque nos interesa comparar en cuantas hojas coinciden los ejemplos.

Como se puede ver, logramos que vuelvan a aparecer los clusters en el grafico.
Ahora podriamos probar alguna técnica de clustering y plotear los resultados usando este ultimo embedding

In [ ]:
%time dbscan = cluster.DBSCAN().fit(X)
plot_data(final_emb, dbscan.labels_, title='DBSCAN clustering', legend=False)

Se puede ver que DBSCAN no es capaz de encontrar clusters, tiene sentido porque por la alta dimensionalidad los puntos tienden a estar todos dispersos, vamos a intentar repetir el truco del arbols

In [ ]:
%time dbscan = cluster.DBSCAN(eps=1).fit(final_emb)
plot_data(final_emb, dbscan.labels_, title='DBSCAN clustering', legend=True)

Podemos ver que usando solo el embedding en 2D, dbscan funciona bien.
Que pasa ahora si tenemos nuevos ejemplos que clusterizar...

In [ ]:
more_characters = load_dd(80_000)
noise_columns = []
for i in range(100):
    col = f'noisy_{i}'
    more_characters[col] = np.random.random(size=len(more_characters))
    noise_columns.append(col)

In [ ]:
X_new = dfm.fit_transform(more_characters)
y_new = more_characters.race

%time new_leaves = et.apply(X_new)
%time new_emb = umap_reducer.transform(new_leaves)
%time final_dbscan = cluster.DBSCAN(eps=1).fit(new_emb)

In [ ]:
plot_data(new_emb, final_dbscan.labels_, title='DBSCAN clustering', legend=True)

* Podemos ver que todo el pipeline anda bastante rapido y bien
* En esta ultima parte del pipeline nunca usamos most_common_races, ergo, podemos clusterizar de forma semi supervisada, o hacer metric learning 

### Conclusiones

* Clusterizar no es una tarea sencilla
* DBSCAN es una buena opción para empezar (o HDBSCAN)
* Si tenemos muchas dimensiones tenemos que tratar de reducirlas primero
* Un buen pipeline puede ser combinando varias técnicas
 * Rule of thumb: **(300+) => PCA => (50) => UMAP (10-20) => DBSCAN / Classification**
 
 



### Gracias! Preguntas?


<div style="float: left;"><img src="../common/imgs/man-qmark.jpg" width="300" align="middle"></div> 

<div>
<div>
  <img src="../common/imgs/gmail-1162901_960_720.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;">arielrossanigo@gmail.com</span>
</div>
<div>
  <img src="../common/imgs/twitter-312464_960_720.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;">@arielrossanigo</span>
</div>
<div>
  <img src="../common/imgs/github-154769__340.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;">https://github.com/arielrossanigo</span>
</div>
<div>
  <img src="../common/imgs/Linkedin_icon.svg" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;">https://www.linkedin.com/in/arielrossanigo/</span>
</div>

</div>
